In [ ]:
import sys
sys.path.append('../scripts')
from dyck_caps import *
from braidgroup import *
from cytoolz import *
from knottedsurface import *
from itertools import product
import pandas as pd
import numpy as np
get=curry(get)

In [ ]:
caps = list(gen_caps(4))
caps = list(map(tuple, caps))
print(len(caps))

In [ ]:
%%timeit
mybraids = braidsNoRepeats(8, 2) #get all braids on 8 strands with exactly 2 crossings.
braids = list(map(get(0), mybraids))
print(len(braids))
print(f'number of potential diagrams is {len(caps)**3 * len(braids)}')

In [ ]:
# potential_triplanes = product(caps, caps, caps, mybraids)
idx = pd.MultiIndex.from_product([caps,caps,caps,braids],names=['c1','c2','c3','b'])
df = pd.DataFrame(index=idx).reset_index()

In [ ]:
#checks a triple (cap,cap,cap,braid) to see if it makes a valid triplane diagram
def tupleCheck(c1:list[tuple[int]], c2:list[tuple[int]], c3:list[tuple[int]], b:tuple[int], *args, **kwargs) -> bool:
    del(args)
    del(kwargs)
    try:
        n = 2*len(c1)
        red = pairs_to_cap(c1)
        blue = pairs_to_cap(c2)
        green = add_cap(tuple_to_braid(b, n),c3)
        return is_triplane(red, blue, green)
    except:
        return None

In [ ]:
df['triplane'] = df.apply(lambda row:tupleCheck(*row[:4]), axis=1, raw=True) #raw=True bypasses pd.Series for row.
# df['triplane'] = np.vectorize(tupleCheck)(df.c1,df.c2,df.c3,df.b)

In [ ]:
%%timeit
good_triples=dict()
samps = df.sample(n=256)
for i in range(256):
    row = samps.iloc[i]
    if bool(tupleCheck(*df.iloc[i])):
        good_triples[i] = (row.c1,row.c2,row.c3,row.b)
    else:
        pass

In [ ]:
len(good_triples)

In [ ]:
good_triples

In [ ]:
#Every 1-crossing thing works.